# Importing the dependencies

In [58]:
import pandas as pd
import numpy as np
import sqlite3

import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")

# Prompt

In [59]:
system_intruction = """You are an expert at converting text/image (from multiple languages) to SQL code. Here, the SQL Database you are getting is called Quotation_Database and 
    has the following columns : \n\n
    
       \t1.	quotation_id (Primary Key)
	\t\t•	Type: INT (Auto-increment)
	\t\t•	Description: Unique identifier for each quotation
	\t2.	vendor_name
	\t\t•	Type: VARCHAR(255)
	\t\t•	Description: Name of the vendor providing the quotation.
	\t3.	quotation_text
	\t\t•	Type: TEXT
	\t\t•	Description: The raw text of the quotation extracted from the WhatsApp chat.
	\t4.	created_at
	\t\t•	Type: DATETIME
	\t\t•	Description: Timestamp when the quotation was created or entered. By default present date and current time
        \t5.item_description
	\t\t•	Type: TEXT
	\t\t•	Description: Description of the items or services quoted.
	\t6.item_quantity
	\t\t•	Type: INT
	\t\t•	Description: Quantity of the items or services quoted.
	\t7.	total_amount
	\t\t•	Type: DECIMAL(10, 2)
	\t\t•	Description: Total amount of the quotation, if identifiable. By default INR
	\t8.	currency
	\t\t•	Type: VARCHAR(10)
	\t\t•	Description: Currency used in the quotation (if identifiable).
	\t9.	status
	\t\t•	Type: ENUM(‘pending’, ‘processed’)
	\t\t•	Description: Status to track if the quotation has been processed into a structured format, be default 'pending'
        
        
    \n\nNow, the above columns which I have mentioned are the columns of the table Quotation_Database and are exactly the same as they are i.e. are case sensitive.
    So, make sure you do not make any mistakes in the column names.\n\n
    
    \n\nNow, in your input, you most likely receive an image or text, from which you will have to extract the data need to convert it into a sql command of adding a new entry as shown in the example below. Since these are strings/numeric values, you have to convert it as it is.\n\n
    
        
   \n\nTake this for an example, Inserting a New Quotation : 

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'ABC Supplies', 
    'Quotation for 10 laptops and 5 monitors', 
    CURRENT_TIMESTAMP, 
    'Laptops, Monitors', 
    15, 
    50000.00, 
    'INR', 
    'pending'
);

 \n\nTake this for an example, Inserting a Inserting Another Quotation : 

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'XYZ Traders', 
    'Price quote for 100 office chairs', 
    CURRENT_TIMESTAMP, 
    'Office Chairs', 
    100, 
    35000.00, 
    'INR', 
    'pending'
);

 \n\nTake this for an example, Inserting a Quotation with a Different Currency : 

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'Global Tech Solutions', 
    'Quote for software licenses for 50 users', 
    CURRENT_TIMESTAMP, 
    'Software Licenses', 
    50, 
    1000.00, 
    'USD', 
    'pending'
);

 \n\nTake this for an example, Inserting a Quotation Marked as Processed :

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'Pro Services Ltd.', 
    'Quote for web development services', 
    CURRENT_TIMESTAMP, 
    'Web Development Services', 
    1, 
    200000.00, 
    'INR', 
    'processed'
);

   Now, in case you do not understand any query or are not able to make out the values, you can act as follows : 

\n\nExample 1: Bot Fails to Extract Item Description, Quantity, and Total Amount : 

\n
INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'ABC Supplies', 
    'Quotation details attached.', 
    CURRENT_TIMESTAMP, 
    NULL, 
    NULL, 
    NULL, 
    'INR', 
    'pending'
);

\n\nExample 2: Only Vendor Name and Quotation Text Extracted, Other Fields Missing : 

\n
INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'XYZ Traders', 
    'Kindly find the quotation below.', 
    CURRENT_TIMESTAMP, 
    NULL, 
    NULL, 
    NULL, 
    'INR', 
    'pending'
);

\n\nExample 3: Missing Vendor Name and Total Amount : 

\n
INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    NULL, 
    'Quotation for various office supplies.', 
    CURRENT_TIMESTAMP, 
    'Office Supplies', 
    50, 
    NULL, 
    'INR', 
    'pending'
);

\n\nExample 4: Bot Only Extracts Quotation Text and Defaults Other Fields : 

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    NULL, 
    'Please review the attached quotation for your review.', 
    CURRENT_TIMESTAMP, 
    NULL, 
    NULL, 
    NULL, 
    'INR', 
    'pending'
);

\n\nExample 5: Bot Only Extracts Vendor Name and Date : 

\n
INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'Pro Services Ltd.', 
    NULL, 
    CURRENT_TIMESTAMP, 
    NULL, 
    NULL, 
    NULL, 
    'INR', 
    'pending'
);
      
    \n\nAlso, make sure that the SQL code / stirng output does not have ``` in the beginning or the end of your answer, and it should'nt even have "" anywhere in the beginning or end of the answer.
    Also, the word "SQL" or any other words should not be present in your output, apart from the relevant format. Even the symbol of skipping a line
    like should not be present in your output. Just the commands or output as you have been shown above in the examples.
    Just the commands or output as you have been shown above in the examples.
"""

# Database Creation

In [60]:
connection = sqlite3.connect("../Data/database.db")
cursor = connection.cursor()

table_info = """ 
CREATE TABLE quotations (
    quotation_id INTEGER PRIMARY KEY AUTOINCREMENT,  -- Unique identifier for each quotation
    vendor_name TEXT NOT NULL,                       -- Name of the vendor providing the quotation
    quotation_text TEXT NOT NULL,                    -- Raw text of the quotation from the WhatsApp chat
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,   -- Timestamp when the quotation was created
    item_description TEXT,                           -- Description of the items or services quoted
    item_quantity INTEGER,                           -- Quantity of the items or services quoted
    total_amount DECIMAL(10, 2),                     -- Total amount of the quotation, if identifiable
    currency TEXT DEFAULT 'INR',                     -- Currency used in the quotation, defaulting to 'INR'
    status TEXT DEFAULT 'pending'                     -- Status to track if the quotation has been processed, defaulting to 'pending'
);
"""

cursor.execute(table_info)

In [61]:
connection.commit()

# OpenAI Instance Creation

In [62]:
client = OpenAI(api_key=KEY)

In [63]:
def get_response(client,system_intruction,text):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_intruction},
            {
                "role": "user",
                "content": text
            }
        ]
    )
    return completion.choices[0].message.content

# Inferencing

In [64]:
examples = [
    """
        Sharma Electric:
        Hello, 
        We are quoting for the electrical supplies:
        100 meters of wire - ₹20 per meter
        5 switches - ₹150 each
        Total: ₹2500
        Let us know if any changes are required. Thanks!
    """,
    """
        Rajesh Construction Materials:
        Good evening, 
        Here’s the quote for construction materials:
        Bricks - 1000 pcs @ ₹5 per brick
        Cement - 50 bags @ ₹320 per bag
        Total: ₹26500
        We are ready for immediate delivery.
    """,
    """
        Patel Hardware:
            Dear Sir/Madam,
            Quoting for hardware tools:
            20 hammers - ₹500 each
            50 screwdrivers - ₹150 each
            Total: ₹17500
            Let us know if any adjustments are needed. Thank you.
    """,
    """ 
        गुप्ता इलेक्ट्रिक:
        नमस्कार,
        यहाँ आपके ऑर्डर के लिए इलेक्ट्रिक सामान की कोटेशन है:
        100 मीटर तार - ₹20 प्रति मीटर
        5 स्विच - ₹150 प्रति स्विच
        कुल: ₹2500
        धन्यवाद, कृपया पुष्टि करें।
    """,
    """ 
        ರಾಮಚಂದ್ರ ಎಂಟರ್‌ಪ್ರೈಸಸ್:
        ಸುಪ್ರಭಾತ,
        ಕಚೇರಿ ಪೀಠೋಪಕರಣಗಳ ಕ್ವೊಟೇಶನ್ ಇಲ್ಲಿದೆ:
        50 ಪ್ಲಾಸ್ಟಿಕ್ ಕೂರಚಿಗಳು - ₹450 ಪ್ರತಿ ಕೂರಚಿ
        10 ಮೇಜುಗಳು - ₹2500 ಪ್ರತಿ ಮೇಜು
        ಒಟ್ಟು: ₹52500
        ಪಾವತಿ ಷರತ್ತು: 50% ಮುಂಗಡ.
    """
]

In [75]:
print(get_response(client,system_intruction,examples[0]))

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'Sharma Electric', 
    'We are quoting for the electrical supplies: 100 meters of wire - ₹20 per meter, 5 switches - ₹150 each. Total: ₹2500. Let us know if any changes are required. Thanks!', 
    CURRENT_TIMESTAMP, 
    'Wire, Switches', 
    105, 
    2500.00, 
    'INR', 
    'pending'
);


In [49]:
print(get_response(client,system_intruction,examples[1]))

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'Rajesh Construction Materials', 
    'Good evening, Here’s the quote for construction materials: Bricks - 1000 pcs @ ₹5 per brick Cement - 50 bags @ ₹320 per bag Total: ₹26500 We are ready for immediate delivery.', 
    CURRENT_TIMESTAMP, 
    'Bricks, Cement', 
    1050, 
    26500.00, 
    'INR', 
    'pending'
);



In [50]:
print(get_response(client,system_intruction,examples[2]))

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'Patel Hardware', 
    'Quoting for hardware tools:\n20 hammers - ₹500 each\n50 screwdrivers - ₹150 each\nTotal: ₹17500\nLet us know if any adjustments are needed. Thank you.', 
    CURRENT_TIMESTAMP, 
    'Hammers, Screwdrivers', 
    70, 
    17500.00, 
    'INR', 
    'pending'
);


In [51]:
print(get_response(client,system_intruction,examples[3]))

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'गुप्ता इलेक्ट्रिक', 
    'नमस्कार, यहाँ आपके ऑर्डर के लिए इलेक्ट्रिक सामान की कोटेशन है: 100 मीटर तार - ₹20 प्रति मीटर 5 स्विच - ₹150 प्रति स्विच कुल: ₹2500 धन्यवाद, कृपया पुष्टि करें।', 
    CURRENT_TIMESTAMP, 
    'तार, स्विच', 
    105, 
    2500.00, 
    'INR', 
    'pending'
);


In [52]:
print(get_response(client,system_intruction,examples[4]))

INSERT INTO quotations (
    vendor_name, 
    quotation_text, 
    created_at, 
    item_description, 
    item_quantity, 
    total_amount, 
    currency, 
    status
) VALUES (
    'ರಾಮಚಂದ್ರ ಎಂಟರ್‌ಪ್ರೈಸಸ್', 
    'ಸುಪ್ರಭಾತ, ಕಚೇರಿ ಪೀಠೋಪಕರಣಗಳ ಕ್ವೊಟೇಶನ್ ಇಲ್ಲಿದೆ: 50 ಪ್ಲಾಸ್ಟಿಕ್ ಕೂರಚಿಗಳು - ₹450 ಪ್ರತಿ ಕೂರಚಿ, 10 ಮೇಜುಗಳು - ₹2500 ಪ್ರತಿ ಮೇಜು, ಒಟ್ಟು: ₹52500, ಪಾವತಿ ಷರತ್ತು: 50% ಮುಂಗಡ.', 
    CURRENT_TIMESTAMP, 
    'Plastic Chairs, Tables', 
    60, 
    52500.00, 
    'INR', 
    'pending'
);


# Inserting in Database

In [72]:
def run_sql_query(text):
    sql_query = get_response(client,system_intruction,text)
    cursor.execute(sql_query)
    connection.commit()

In [73]:
for example in examples:
    run_sql_query(example)